# 🚀 Système de Q&A Ultra-Rapide sur Transcription Vocale

Ce notebook implémente un système de questions-réponses rapide basé sur :
- **Sentence Transformers** : Embeddings multilingues français
- **FAISS** : Recherche vectorielle ultra-rapide
- Temps de réponse : ~50-200ms

---

## 📦 1. Installation des dépendances

In [1]:
!pip install sentence-transformers faiss-cpu numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 1.3 MB/s eta 0:00:000:00:010:00:01:010m


In [2]:
import numpy as np
from sentence_transformers import SentenceTransformer
import faiss
from typing import List, Tuple
import re
import time
import random

W1218 09:00:01.907000 87432 site-packages/torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


## 🔧 2. Classe FastQASystem

In [3]:
class FastQASystem:
    """Système de Q&A ultra-rapide basé sur la recherche vectorielle"""
    
    def __init__(self, model_name: str = 'paraphrase-multilingual-MiniLM-L12-v2'):
        """
        Initialise le système avec un modèle d'embeddings.
        
        Args:
            model_name: Nom du modèle SentenceTransformer
                       Options: 
                       - 'paraphrase-multilingual-MiniLM-L12-v2' (rapide, bon)
                       - 'paraphrase-multilingual-mpnet-base-v2' (plus lent, meilleur)
        """
        print(f"📦 Chargement du modèle {model_name}...")
        self.model = SentenceTransformer(model_name)
        self.index = None
        self.segments = []
        print("✓ Modèle chargé!")
        
    def prepare_transcription(self, transcription: str, window_size: int = 1) -> List[str]:
        """
        Découpe la transcription en segments avec contexte.
        
        Args:
            transcription: Texte de la transcription
            window_size: Nombre de phrases avant/après à inclure pour le contexte
            
        Returns:
            Liste de segments avec contexte
        """
        # Découpe par phrases
        sentences = re.split(r'[.!?]+', transcription)
        sentences = [s.strip() for s in sentences if s.strip()]
        
        segments = []
        
        # Crée des segments avec fenêtre de contexte
        for i, sentence in enumerate(sentences):
            context = []
            
            # Ajoute les phrases précédentes
            for j in range(max(0, i - window_size), i):
                context.append(sentences[j])
            
            # Phrase actuelle
            context.append(sentence)
            
            # Ajoute les phrases suivantes
            for j in range(i + 1, min(len(sentences), i + window_size + 1)):
                context.append(sentences[j])
            
            segments.append(' '.join(context))
        
        return segments
    
    def index_transcription(self, transcription: str, window_size: int = 1):
        """
        Indexe la transcription pour recherche rapide.
        
        Args:
            transcription: Texte de la transcription
            window_size: Taille de la fenêtre de contexte
        """
        print("\n🔄 Préparation des segments...")
        self.segments = self.prepare_transcription(transcription, window_size)
        print(f"   → {len(self.segments)} segments créés")
        
        print("🔄 Encodage des segments...")
        start = time.time()
        embeddings = self.model.encode(
            self.segments, 
            show_progress_bar=True,
            convert_to_numpy=True
        )
        print(f"   → Encodage terminé en {time.time() - start:.2f}s")
        
        print("🔄 Création de l'index FAISS...")
        dimension = embeddings.shape[1]
        self.index = faiss.IndexFlatIP(dimension)  # Inner Product pour similarité cosinus
        
        # Normalise pour utiliser la similarité cosinus
        faiss.normalize_L2(embeddings)
        self.index.add(embeddings)
        
        print(f"✓ Indexation terminée! ({self.index.ntotal} vecteurs)")
    
    def search(self, question: str, top_k: int = 3) -> List[Tuple[str, float]]:
        """
        Recherche les segments les plus pertinents.
        
        Args:
            question: Question posée
            top_k: Nombre de résultats à retourner
            
        Returns:
            Liste de tuples (segment, score de similarité)
        """
        if self.index is None:
            raise ValueError("Le système n'a pas été indexé. Appelle d'abord index_transcription()")
        
        # Encode la question
        question_embedding = self.model.encode([question], convert_to_numpy=True)
        faiss.normalize_L2(question_embedding)
        
        # Recherche dans l'index
        scores, indices = self.index.search(question_embedding, top_k)
        
        results = []
        for idx, score in zip(indices[0], scores[0]):
            results.append((self.segments[idx], float(score)))
        
        return results
    
    def answer(self, question: str, min_confidence: float = 0.3) -> dict:
        """
        Répond à la question de manière naturelle.
        
        Args:
            question: Question posée
            min_confidence: Score minimum pour considérer la réponse valide
            
        Returns:
            Dictionnaire avec 'answer', 'confidence', 'time_ms'
        """
        start = time.time()
        
        results = self.search(question, top_k=1)
        
        elapsed_ms = (time.time() - start) * 1000
        
        if not results:
            return {
                'answer': "Désolé, je n'ai pas trouvé d'information sur ce sujet.",
                'confidence': 0.0,
                'time_ms': elapsed_ms
            }
        
        best_match, score = results[0]
        
        if score < min_confidence:
            return {
                'answer': "Hmm, je ne suis pas sûr d'avoir compris ta question. Peux-tu reformuler ?",
                'confidence': score,
                'time_ms': elapsed_ms
            }
        
        answer = self._format_answer(best_match, score)
        
        return {
            'answer': answer,
            'confidence': score,
            'time_ms': elapsed_ms,
            'raw_segment': best_match
        }
    
    def _format_answer(self, text: str, confidence: float) -> str:
        """Ajoute un peu de naturel à la réponse"""
        text = text.strip()
        
        if confidence > 0.8:
            prefixes = ["Voilà : ", "Ah oui ! ", "Exactement : ", ""]
        elif confidence > 0.5:
            prefixes = ["D'après ce que j'ai : ", "Il semblerait que : ", ""]
        else:
            prefixes = ["Je pense que : ", "Peut-être que : "]
        
        prefix = random.choice(prefixes)
        return f"{prefix}{text}"

## 📝 3. Exemple de transcription

In [4]:
# Transcription d'exemple (remplace par ta vraie transcription)
transcription_exemple = """
John est arrivé ce matin avec son bob rose flashy, cette sacré canaille. 
Tout le monde a rigolé en le voyant. 
Il a dit que c'était un cadeau de sa grand-mère Josette.
Marie portait une casquette bleue marine très élégante.
Le bob de John est vraiment voyant, on ne peut pas le manquer.
On l'a surnommé le flamant rose après ça.
Sophie a apporté des croissants pour tout le monde.
Les croissants venaient de la boulangerie du coin de la rue.
Pierre a raconté une blague mais personne n'a ri.
Il était un peu vexé mais il a fait semblant de rire aussi.
La réunion a commencé à 9h précises.
Le directeur a annoncé de bonnes nouvelles sur les ventes du trimestre.
"""

print("📄 Transcription chargée!")
print(f"Longueur: {len(transcription_exemple)} caractères")

📄 Transcription chargée!
Longueur: 663 caractères


## 🚀 4. Initialisation et indexation

In [5]:
# Initialise le système
qa_system = FastQASystem()

📦 Chargement du modèle paraphrase-multilingual-MiniLM-L12-v2...


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✓ Modèle chargé!


In [6]:
# Indexe la transcription
qa_system.index_transcription(transcription_exemple, window_size=1)


🔄 Préparation des segments...
   → 12 segments créés
🔄 Encodage des segments...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

   → Encodage terminé en 0.97s
🔄 Création de l'index FAISS...
✓ Indexation terminée! (12 vecteurs)


## 💬 5. Test avec des questions

In [7]:
# Liste de questions de test
questions_test = [
    "De quelle couleur est le bob de John ?",
    "Qui porte une casquette ?",
    "Quel surnom a été donné à John ?",
    "Qui a apporté des croissants ?",
    "D'où viennent les croissants ?",
    "À quelle heure a commencé la réunion ?",
    "Qu'a annoncé le directeur ?",
]

In [8]:
# Test des questions
print("="*70)
print("🧪 TEST DU SYSTÈME DE Q&A")
print("="*70 + "\n")

for question in questions_test:
    result = qa_system.answer(question)
    
    print(f"❓ Q: {question}")
    print(f"💬 R: {result['answer']}")
    print(f"📊 Confiance: {result['confidence']:.2%}")
    print(f"⚡ Temps: {result['time_ms']:.0f}ms")
    print("-" * 70 + "\n")

🧪 TEST DU SYSTÈME DE Q&A

❓ Q: De quelle couleur est le bob de John ?
💬 R: D'après ce que j'ai : John est arrivé ce matin avec son bob rose flashy, cette sacré canaille Tout le monde a rigolé en le voyant
📊 Confiance: 60.32%
⚡ Temps: 695ms
----------------------------------------------------------------------

❓ Q: Qui porte une casquette ?
💬 R: Je pense que : Marie portait une casquette bleue marine très élégante Le bob de John est vraiment voyant, on ne peut pas le manquer On l'a surnommé le flamant rose après ça
📊 Confiance: 43.25%
⚡ Temps: 337ms
----------------------------------------------------------------------

❓ Q: Quel surnom a été donné à John ?
💬 R: Il semblerait que : John est arrivé ce matin avec son bob rose flashy, cette sacré canaille Tout le monde a rigolé en le voyant Il a dit que c'était un cadeau de sa grand-mère Josette
📊 Confiance: 56.27%
⚡ Temps: 14ms
----------------------------------------------------------------------

❓ Q: Qui a apporté des croissants ?
💬 R

## 🔍 6. Recherche avancée (top 3 résultats)

In [9]:
# Exemple de recherche avec plusieurs résultats
question = "Parle-moi de John"
results = qa_system.search(question, top_k=3)

print(f"🔍 Question: {question}\n")
print("Top 3 résultats:\n")

for i, (segment, score) in enumerate(results, 1):
    print(f"{i}. Score: {score:.2%}")
    print(f"   {segment}\n")

🔍 Question: Parle-moi de John

Top 3 résultats:

1. Score: 62.43%
   John est arrivé ce matin avec son bob rose flashy, cette sacré canaille Tout le monde a rigolé en le voyant

2. Score: 56.93%
   John est arrivé ce matin avec son bob rose flashy, cette sacré canaille Tout le monde a rigolé en le voyant Il a dit que c'était un cadeau de sa grand-mère Josette

3. Score: 49.57%
   Le bob de John est vraiment voyant, on ne peut pas le manquer On l'a surnommé le flamant rose après ça Sophie a apporté des croissants pour tout le monde



## 🎯 7. Test interactif

In [10]:
# Pose ta propre question ici
ma_question = "De quelle couleur est le bob de John ?"  # Modifie cette ligne

result = qa_system.answer(ma_question)

print(f"❓ Question: {ma_question}")
print(f"💬 Réponse: {result['answer']}")
print(f"\n📊 Détails:")
print(f"   - Confiance: {result['confidence']:.2%}")
print(f"   - Temps: {result['time_ms']:.0f}ms")
if 'raw_segment' in result:
    print(f"   - Segment source: {result['raw_segment']}")

❓ Question: De quelle couleur est le bob de John ?
💬 Réponse: John est arrivé ce matin avec son bob rose flashy, cette sacré canaille Tout le monde a rigolé en le voyant

📊 Détails:
   - Confiance: 60.32%
   - Temps: 53ms
   - Segment source: John est arrivé ce matin avec son bob rose flashy, cette sacré canaille Tout le monde a rigolé en le voyant


## 📊 8. Benchmark de performance

In [11]:
# Test de performance sur plusieurs questions
import statistics

times = []
for _ in range(20):
    for q in questions_test:
        result = qa_system.answer(q)
        times.append(result['time_ms'])

print("📊 STATISTIQUES DE PERFORMANCE")
print("=" * 40)
print(f"Temps moyen:  {statistics.mean(times):.1f}ms")
print(f"Temps médian: {statistics.median(times):.1f}ms")
print(f"Temps min:    {min(times):.1f}ms")
print(f"Temps max:    {max(times):.1f}ms")
print(f"\n🎯 Total de {len(times)} requêtes testées")

📊 STATISTIQUES DE PERFORMANCE
Temps moyen:  12.8ms
Temps médian: 12.8ms
Temps min:    9.8ms
Temps max:    60.1ms

🎯 Total de 140 requêtes testées
